# Tutorial 1: "Moving Pictures"

For the in class portion of our qiime2 primer, we'll be working through the "Moving Pictures" tutorial published on the qiime2 docs (https://amplicon-docs.qiime2.org/en/latest/tutorials/moving-pictures.html). 

We'll be using a small dataset of samples collected across from two individuals who took antibiotics. Each individual was sampled at four body sites, four times, over six months, beginning with the antibiotic exposure. 

We're going to practice processing raw read data into ASVs, creating a phylogeny, running diversity analysis, and testing for differential abundance. 

## 1) Load the dataset into QIIME

**Step 1: load sample metadata**

In [ ]:
from qiime2 import Metadata
import requests
## download the metadata file from the online source
url = 'https://moving-pictures-tutorial.readthedocs.io/en/latest/data/moving-pictures/sample-metadata.tsv'
fn = 'sample-metadata.tsv'
r= requests.get(url)
with open(fn, "wb") as f:
    f.write(r.content)
### remove dashes from column names 
import pandas as pd
pd.read_csv('sample-metadata.tsv',sep='\t',index_col=0).rename(columns = {'reported-antibiotic-usage':\
     'reported_antibiotic_usage', 'days-since-experiment-start': 'days_since_experiment_start'}).to_csv('sample-metadata.tsv', sep = '\t')
## load into qiime2
sample_metadata_md = Metadata.load(fn)

Create an interactive table!
- you can view this at https://view.qiime2.org/

In [ ]:
import qiime2.plugins.metadata.actions as metadata_actions

sample_metadata_viz, = metadata_actions.tabulate(
    input=sample_metadata_md,
)
sample_metadata_viz.save('sample-metadata.qzv')

Alternatively, just export to pandas

In [ ]:
metadata_pandas= sample_metadata_md.to_dataframe()
metadata_pandas.head()

**Step 2: download the sequences**

Our data is pretty old (2011!), so its single-end. Its also still multiplexed, so just one big file.

In [ ]:
import zipfile
## again, use the requests library to download from the online source
url = 'https://moving-pictures-tutorial.readthedocs.io/en/latest/data/moving-pictures/emp-single-end-sequences.zip'
fn = 'emp-single-end-sequences.zip'
r = requests.get(url)
with open(fn, "wb") as f:
    f.write(r.content)
## extra unzipping step 
with zipfile.ZipFile(fn) as zf:
    zf.extractall('emp-single-end-sequences')
## load into a qiime2 data structure, the "artifact"
from qiime2 import Artifact
emp_single_end_sequences = Artifact.import_data(
    'EMPSingleEndSequences',
    'emp-single-end-sequences',
)

## 2) Demultiplex the sequences and processes reads into ASVs 

**Step 1: match reads to samples (demultiplexing)**

In [ ]:
import qiime2.plugins.demux.actions as demux_actions
## each sample has a unique barcode sequence. pull it out of the metadata
barcode_sequence_mdc = sample_metadata_md.get_column('barcode-sequence')
## now feed it to qiime's demultiplexing function
demux, demux_details = demux_actions.emp_single(
    seqs=emp_single_end_sequences,
    barcodes=barcode_sequence_mdc,
)
## summarize the demultiplexing results
demux_viz, = demux_actions.summarize(
    data=demux,
)
## save for export to qiime2 view
demux_viz.save('demux.qzv')

**Step 2: take a look at base quality scores**

Read quality is often lower at the ends. We'll want to remove lower quality bases before creating ASVs. 

Take a look at the file you just created (demux.qzv) at https://view.qiime2.org/. Find the position on the Interactive Quality Plot where quality scores start to get pretty low (ie < 20). 

I think 120 bases is a pretty good spot to trim.

We'll provide our denoising tool (DADA2) with this information and it will slice off the lower quality tails

**Step 3: create ASVs (denoise)**

We're using DADA2 to create our ASVs. DADA2 tries to recover the underlying, true 16S sequences, given the sequencing data and quality scores that we see. 

In our dataset, we have 34 samples, each with ~7,800 reads, all 120 bases long, for a total of ~32 million bases.

Each base has an associated Phred, or "Q", score. They represent error rate as  $Q= -10\log_{10}(P_{\text{error}})$

Therefore, even if we had really high Q-scores of 30 (1/1000 error rate), DADA2 would still look to correct 32 million/ 1000 = 32 thousand errors!

In addition to correcting sequencing errors, DADA2 also seeks identify and remove chimeric sequences produced during PCR. 


In [ ]:
import qiime2.plugins.dada2.actions as dada2_actions
# input: the demultiplexed sequences, trimming parameters
# output: a feature table (samples x ASVs), ASV sequences, and denoising stats (stuff like ASVs/sample)
table, rep_seqs, stats = dada2_actions.denoise_single(
    demultiplexed_seqs=demux,
    trim_left=0,
    trunc_len=120,
)

tabulate statistics on the denoising (and view at view.qiime2)

In [ ]:
stats_dada2_md_md = stats.view(Metadata)
stats_viz, = metadata_actions.tabulate(
    input=stats_dada2_md_md,
)
stats_viz.save('stats_viz.dada2.qzv')

... or just use pandas

In [ ]:
stats_dada2_md_md.to_dataframe().head()

#### *Question: How many non-chimeric reads do we recover per sample, on average?*

*answer here*

## 3) Build a tree and taxonomize 

**Step 1: tree building**


Bacterial 16S sequences have been used for building phylogenies since the '70s. We're going to use MAFFT to create a multiple sequence alignment and FastTree to create a tree. Qiime wraps all of this in a single command. By default, qiime uses midpoint rooting and masks highly variable positions in the MSA before tree building.

In [ ]:
import qiime2.plugins.phylogeny.actions as phylogeny_actions

aligned_rep_seqs, masked_aligned_rep_seqs, unrooted_tree, rooted_tree = phylogeny_actions.align_to_tree_mafft_fasttree(
    sequences=rep_seqs,
)

**Step 2: taxonomizing**

It's standard practice to use a naive bayes classifier to taxonomize 16S ASVs. Under the hood, qiime2 is just using using the scikit-learn python library. There are two main 16S databases, Silva and GreenGenes. GreenGenes is currently more up to date. Folks at qiime2 train and publish classifier weights on the qiime2 site, so all we have to do is download and run their classifier on our data. 

The qiime2 command will automatically provide classifications at all taxonomic levels. By default, it returns classifications with confidences (posterior probability) > 0.7. In the literature, its pretty common to use an NB classifier for kingdom to genus level classifications, and exact string matches for species and below. 

- *Note*: taxonomy and phylogeny are generally treated separately in 16S metagenomic workflows. This means that choice of database will not impact phylogeny-based diversity analyses

Download:

In [ ]:
url = 'https://data.qiime2.org/2021.8/common/gg-13-8-99-515-806-nb-classifier.qza'
fn = 'gg-13-8-99-515-806-nb-classifier.qza'
r = requests.get(url)
with open(fn, "wb") as f:
    f.write(r.content)
gg_13_8_99_515_806_nb_classifier = Artifact.load(fn)


Run: 

In [ ]:
import qiime2.plugins.feature_classifier.actions as feature_classifier_actions
## run the classifier
taxonomy, = feature_classifier_actions.classify_sklearn(
    classifier=gg_13_8_99_515_806_nb_classifier,
    reads=rep_seqs,
)
## export to a version that we can view
taxonomy_as_md_md = taxonomy.view(Metadata)
taxonomy_viz, = metadata_actions.tabulate(
    input=taxonomy_as_md_md,
)

Taxonomies for our 770 ASVs:

In [ ]:
taxonomy_as_md_md.to_dataframe().sort_values(by = 'Confidence')

**Step 3: create the taxa barplot**

In [ ]:
import qiime2.plugins.taxa.actions as taxa_actions

taxa_bar_plots_viz, = taxa_actions.barplot(
    table=table,
    taxonomy=taxonomy,
    metadata=sample_metadata_md,
)
taxa_bar_plots_viz.save('taxa_barplot.qzv')

### *Question: Load the taxa barplot. Color by level 3 taxonomy (class) and sort by body site. Which body site is characterized by high relative abundances of Clostridia?*

*answer here*

## 4) Diversity analyses

**Step 1: create an alpha rarefaction plot**

Richness (how many ASVs we see in a sample) and other alpha diversity measures are dependent on sampling depth. This function will downsample each sample and plot alpha diversity as a function of sampling depth.

Later on, we'll want to compare samples at equal sampling depth. We use the rarefaction plot to find a downsampling depth where we see as much diversity as possible, while keeping as many samples as possible. 

Since this function calculates alpha diversity metrics repeatedly at different sampling depths, its a little slow.

In [ ]:
import qiime2.plugins.diversity.actions as diversity_actions

alpha_rarefaction_viz, = diversity_actions.alpha_rarefaction(
    table=table,
    phylogeny=rooted_tree,
    max_depth=4000,
    metadata=sample_metadata_md,
)


Save and view at view.qiime2

In [ ]:
alpha_rarefaction_viz.save('alpha_rarefaction_viz.qzv')

I think downsampling to 1103 should be ok! Alpha diversity seems to plateau and we only lose 3 samples.

### *Question: Which body sites host the most and least diverse microbiota?*

*Answer here*

**Step 2: calculate all diversity metrics on the downsampled dataset**
- again, qiime2 wraps it all in one command

In [ ]:
action_results = diversity_actions.core_metrics_phylogenetic(
    phylogeny=rooted_tree,
    table=table,
    sampling_depth=1103,
    metadata=sample_metadata_md,
)

**Step 3: extract the metrics that we're interested in**

In [ ]:
# the downsampled dataset
rarefied_table = action_results.rarefied_table
# shannon diversity (alpha diversity)
shannon_vector = action_results.shannon_vector
# weighted unifrac (beta diversity)
weighted_unifrac_distance_matrix = action_results.weighted_unifrac_distance_matrix
# the weighted and unweightedunifrac pcoa plots
unweighted_unifrac_emperor_viz = action_results.unweighted_unifrac_emperor
weighted_unifrac_emperor_viz = action_results.weighted_unifrac_emperor
unweighted_unifrac_emperor_viz.save('unweighted_unifrac_emperor_viz.qzv')
weighted_unifrac_emperor_viz.save('weighted_unifrac_emperor_viz.qzv')

**Step 4: test for associations with metadata**

looks like shannon diversity actually *increased* after antibiotic usage. Pretty cool!
- since each sample has a single number for shannon diversity, we're using a Kruskal-Wallis test

In [ ]:
shannon_group_significance_viz, = diversity_actions.alpha_group_significance(
    alpha_diversity=shannon_vector,
    metadata=sample_metadata_md,
)
shannon_group_significance_viz.save('shannon_significance.qzv')

beta diversity appears to be associated with body site. It makes sense that different bugs would live in different places
- since beta diversity creates a distance matrix, we're using a PERMANOVA
- take a look at the beta diversity pcoa plots (emperor_viz) at view.qiime2!

In [ ]:
body_site_mdc = sample_metadata_md.get_column('body-site')
weighted_unifrac_body_site_group_significance_viz, = diversity_actions.beta_group_significance(
    distance_matrix=weighted_unifrac_distance_matrix,
    metadata=body_site_mdc,
    pairwise=True,
)
subject_mdc = sample_metadata_md.get_column('subject')
weighted_unifrac_subject_group_significance_viz, = diversity_actions.beta_group_significance(
    distance_matrix=weighted_unifrac_distance_matrix,
    metadata=subject_mdc,
    pairwise=True,
)
weighted_unifrac_body_site_group_significance_viz.save('body_site.permanova.qzv')
weighted_unifrac_subject_group_significance_viz.save('subject.permanova.qzv')

## 5) Differential abundance testing

QIIME2 has support for ANCOM, so we're going to use ANCOM. If you're savvy, you can export your feature table to .tsv and use any method you want though. 


As written, this code tests for the effect of antibiotics on the gut microbiome.

**Step 1: create a filtered table**

In [ ]:
import qiime2.plugins.feature_table.actions as feature_table_actions

gut_table, = feature_table_actions.filter_samples(
    table=table,
    metadata=sample_metadata_md,
    where='[body-site]="gut"',
)

**Step 2: test for differential abundance**

First, run on the ASV level

In [ ]:
import qiime2.plugins.composition.actions as composition_actions
## run ancom (formula='reported_antibiotic_usage' tells it to compare microbes between antibiotic and non-antibiotic samples)
ancombc_abx, = composition_actions.ancombc(
    table=gut_table,
    metadata=sample_metadata_md,
    formula='reported_antibiotic_usage',
)
## create a barplot of significant features
da_barplot_abx_viz, = composition_actions.da_barplot(
    data=ancombc_abx,
    significance_threshold=0.1,
)

da_barplot_abx_viz.save('ancom.asv.qzv')

Attach a taxonomy to the top ASV

In [ ]:
# export the ancom results
da_barplot_abx_viz.export_data('ancombc_export')
# read in the p values and log fold changes
ancom_results = pd.read_csv('ancombc_export/q_val_slice.csv', sep=',', index_col=0)[['reported_antibiotic_usageYes'\
                ]].rename(columns={'reported_antibiotic_usageYes':'qval'}).join(pd.read_csv('ancombc_export/lfc_slice.csv', sep=',',
                     index_col=0)[['reported_antibiotic_usageYes'\
                ]].rename(columns={'reported_antibiotic_usageYes':'LFC'})).sort_values(by = 'qval').join(taxonomy_as_md_md.to_dataframe())
# print the most significant ASV
print(ancom_results.iloc[0])

**Step 3: try again at higher taxonomic levels**

In [ ]:
## create a genus-level feature table
gut_table_l6, = taxa_actions.collapse(
    table=gut_table,
    taxonomy=taxonomy,
    level=6,
)
## run ancom
ancombc_abx_l6, = composition_actions.ancombc(
    table=gut_table_l6,
    metadata=sample_metadata_md,
    formula='reported_antibiotic_usage',
)
## plot
da_barplot_abx_l6_viz, = composition_actions.da_barplot(
    data=ancombc_abx_l6,
    significance_threshold=0.1,
)
da_barplot_abx_l6_viz.save('ancom.l6.qzv')

### *Question: Run ANCOM again at the class level (level 3). Are there any differentially abundant taxa?*

*Answer here*

In [ ]:
## create a genus-level feature table
gut_table_l6, = taxa_actions.collapse(
    table=gut_table,
    taxonomy=taxonomy,
    level=3,
)
## run ancom
ancombc_abx_l6, = composition_actions.ancombc(
    table=gut_table_l6,
    metadata=sample_metadata_md,
    formula='reported_antibiotic_usage',
)
## plot
da_barplot_abx_l6_viz, = composition_actions.da_barplot(
    data=ancombc_abx_l6,
    significance_threshold=0.1,
)
da_barplot_abx_l6_viz.save('ancom.l6.qzv')